In [2]:
import pandas as pd


In [5]:
df1 = pd.read_csv("/Users/hushiqi/Desktop/Data/btc_exchange_netflow_usd.csv")
df2 = pd.read_csv("/Users/hushiqi/Desktop/Data/btc_active_addresses.csv")
df3 = pd.read_csv("/Users/hushiqi/Desktop/Data/btc_ohlcv_tx_size_fee_pressure.csv")
df4 = pd.read_csv("/Users/hushiqi/Desktop/Data/miner_share_fees_minute.csv")
df5 = pd.read_csv("/Users/hushiqi/Desktop/Data/Totol_fees.csv")
df6 = pd.read_csv("/Users/hushiqi/Desktop/Data/Exchange_to_WalletWallet_to_Exchange.csv")


In [7]:
print(df1.head())
print(df2.head())
print(df3.head())
print(df4.head())
print(df5.head())
print(df6.head())

                      datetime  exchange_netflow_usd
0  2024-11-05 00:06:00.000 UTC          4.227801e+05
1  2024-11-05 00:18:00.000 UTC          3.890704e+06
2  2024-11-05 00:57:00.000 UTC         -1.487192e+07
3  2024-11-05 00:58:00.000 UTC         -5.606578e+07
4  2024-11-05 01:04:00.000 UTC          1.416559e+06
                      datetime  active_sending_addresses  \
0  2024-11-05 00:06:00.000 UTC                      4430   
1  2024-11-05 00:18:00.000 UTC                      5728   
2  2024-11-05 00:57:00.000 UTC                      4466   
3  2024-11-05 00:58:00.000 UTC                      5356   
4  2024-11-05 01:04:00.000 UTC                      5177   

   active_receiving_addresses  
0                        8288  
1                        7632  
2                        9706  
3                        6027  
4                        7503  
                      datetime  onchain_volume_usd  avg_tx_size_usd  \
0  2024-11-05 00:00:00.000 UTC                 0.0        

In [9]:
def to_minute_index(df, time_col, tz='UTC', how='mean'):
    t = pd.to_datetime(df[time_col], utc=True, errors='coerce')
    df = df.copy()
    df.index = t

    if time_col in df.columns:
        df = df.drop(columns=[time_col])

    df.index = df.index.floor('T')

    if how == 'mean':
        df = df.groupby(df.index).mean(numeric_only=True)
    elif how == 'sum':
        df = df.groupby(df.index).sum(numeric_only=True)
    elif how == 'last':
        df = df.groupby(df.index).last()
    elif how == 'first':
        df = df.groupby(df.index).first()
    else:
        raise ValueError("how must be one of: mean, sum, last, first")

    if df.index.tz is None:
        df.index = df.index.tz_localize(tz)
    else:
        df.index = df.index.tz_convert(tz)

    return df

d1 = to_minute_index(df1, 'datetime')
d2 = to_minute_index(df2, 'datetime')
d3 = to_minute_index(df3, 'datetime')
d4 = to_minute_index(df4, 'minute')
d5 = to_minute_index(df5, 'time')
d6 = to_minute_index(df6, 'minute_bucket')

d1 = d1.rename(columns={'exchange_netflow_usd': 'btc_exchange_netflow_usd'})
d4 = d4.rename(columns={'miner_fee_share_minute': 'miner_fee_share_minute'})
d5 = d5.rename(columns={'total_fees': 'total_fees_btc'})
d6 = d6.rename(columns={'exchange_to_wallet': 'exchange_to_wallet_usd',
                        'wallet_to_exchange': 'wallet_to_exchange_usd'})

start = pd.Timestamp('2024-11-05 00:00:00', tz='UTC')
end   = pd.Timestamp('2025-10-12 23:59:00', tz='UTC')
master_index = pd.date_range(start, end, freq='T', tz='UTC')

frames = []
for d in [d1, d2, d3, d4, d5, d6]:
    frames.append(d.reindex(master_index))

merged = pd.concat(frames, axis=1)

print(merged.shape, merged.index[0], merged.index[-1])
print(merged.head(10))
merged.to_csv("/Users/hushiqi/Desktop/Data/dune_btc.csv",
              index_label='minute_utc')


/var/folders/rp/9x2w7x0j5y75c43279sr0k2m0000gn/T/ipykernel_49451/2059075982.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df.index = df.index.floor('T')
/var/folders/rp/9x2w7x0j5y75c43279sr0k2m0000gn/T/ipykernel_49451/2059075982.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df.index = df.index.floor('T')
/var/folders/rp/9x2w7x0j5y75c43279sr0k2m0000gn/T/ipykernel_49451/2059075982.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df.index = df.index.floor('T')
/var/folders/rp/9x2w7x0j5y75c43279sr0k2m0000gn/T/ipykernel_49451/2059075982.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df.index = df.index.floor('T')
/var/folders/rp/9x2w7x0j5y75c43279sr0k2m0000gn/T/ipykernel_49451/2059075982.py:9: FutureWarning: 'T' is deprecated and will be removed in a 

(492480, 14) 2024-11-05 00:00:00+00:00 2025-10-12 23:59:00+00:00
                           btc_exchange_netflow_usd  active_sending_addresses  \
2024-11-05 00:00:00+00:00                       NaN                       NaN   
2024-11-05 00:01:00+00:00                       NaN                       NaN   
2024-11-05 00:02:00+00:00                       NaN                       NaN   
2024-11-05 00:03:00+00:00                       NaN                       NaN   
2024-11-05 00:04:00+00:00                       NaN                       NaN   
2024-11-05 00:05:00+00:00                       NaN                       NaN   
2024-11-05 00:06:00+00:00             422780.100706                    4430.0   
2024-11-05 00:07:00+00:00                       NaN                       NaN   
2024-11-05 00:08:00+00:00                       NaN                       NaN   
2024-11-05 00:09:00+00:00                       NaN                       NaN   

                           active_receiving